# Attribute

**Original Work**: *Maziar Raissi, Paris Perdikaris, and George Em Karniadakis*

**Github Repo** : https://github.com/maziarraissi/PINNs

**Link:** https://github.com/maziarraissi/PINNs/tree/master/appendix/continuous_time_identification%20(Burgers)

@article{raissi2017physicsI,
  title={Physics Informed Deep Learning (Part I): Data-driven Solutions of Nonlinear Partial Differential Equations},
  author={Raissi, Maziar and Perdikaris, Paris and Karniadakis, George Em},
  journal={arXiv preprint arXiv:1711.10561},
  year={2017}
}

@article{raissi2017physicsII,
  title={Physics Informed Deep Learning (Part II): Data-driven Discovery of Nonlinear Partial Differential Equations},
  author={Raissi, Maziar and Perdikaris, Paris and Karniadakis, George Em},
  journal={arXiv preprint arXiv:1711.10566},
  year={2017}
}

## Libraries and Dependencies

In [1]:
!pip install pyDOE

  Preparing metadata (setup.py) ... done
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=18170 sha256=cbcaa2619a7724eceff3e7661a29319ed7113c0df881a7c5bba7bd055d33ee32
  Stored in directory: /root/.cache/pip/wheels/84/20/8c/8bd43ba42b0b6d39ace1219d6da1576e0dac81b12265c4762e
Successfully built pyDOE


In [2]:
import sys
sys.path.insert(0, '../Utilities/')
import torch
from collections import OrderedDict
from pyDOE import lhs
import numpy as np
import random
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import time
import warnings

warnings.filterwarnings('ignore')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Physics-informed Neural Networks

In [3]:
def set_seed(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False    

In [4]:
seed = 1234
set_seed(seed)

In [5]:
def figsize(scale, nplots = 1):
    fig_width_pt = 390.0                 
    inches_per_pt = 1.0 / 72.27               
    golden_mean = (np.sqrt(5.0)- 1.0 ) / 2.0    
    fig_width = fig_width_pt * inches_per_pt*scale
    fig_height = nplots * fig_width  *golden_mean
    fig_size = [fig_width,fig_height]
    return fig_size

def newfig(width, nplots = 1):
    fig = plt.figure(figsize=figsize(width, nplots))
    ax = fig.add_subplot(111)
    return fig, ax

def savefig(filename, crop = True):
    if crop == True:
        plt.savefig('{}.pdf'.format(filename), bbox_inches='tight', pad_inches=0)
        plt.savefig('{}.eps'.format(filename), bbox_inches='tight', pad_inches=0)
    else:
        plt.savefig('{}.pdf'.format(filename))
        plt.savefig('{}.eps'.format(filename))

In [6]:
# the deep neural network
class DNN(torch.nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        
        # parameters
        self.depth = len(layers) - 1
        
        # set up layer order dict
        self.activation = torch.nn.Tanh
        
        layer_list = list()
        for i in range(self.depth - 1): 
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation()))
            
        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        layerDict = OrderedDict(layer_list)
        
        # deploy layers
        self.layers = torch.nn.Sequential(layerDict)
        
    def forward(self, x):
        out = self.layers(x)
        return out

In [7]:
# the physics-guided neural network
class PhysicsInformedNN(torch.nn.Module):
    def __init__(self, X_u, u, X, layers, lb, ub):
        super(PhysicsInformedNN, self).__init__()
        
        # boundary conditions
        self.lb = torch.tensor(lb).float().to(device)
        self.ub = torch.tensor(ub).float().to(device)
        
        # data
        self.x_u = torch.tensor(X_u[:, 0:1], requires_grad=True).float().to(device)
        self.y_u = torch.tensor(X_u[:, 1:2], requires_grad=True).float().to(device)
        self.t_u = torch.tensor(X_u[:, 2:3], requires_grad=True).float().to(device)
        self.x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        self.y = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)
        self.t = torch.tensor(X[:, 2:3], requires_grad=True).float().to(device)
        self.u = torch.tensor(u).float().to(device)
        self.layers = layers
        # deep neural networks
        self.dnn = DNN(layers).to(device)
        
        # optimizers: using the same settings
        self.optimizer_Adam = torch.optim.Adam(self.dnn.parameters(), lr=0.0005)
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(), 
            lr=1.0, 
            max_iter=50000, 
            max_eval=50000, 
            history_size=50,
            tolerance_grad=1e-6,
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe"
        )
        self.iter = 0
        self.loss_history = []
    def net_u(self, x,y,t):  
        u = self.dnn(torch.cat([x,y,t], dim=1))
        return u
    
    def net_f(self, x, y, t):
        """ The pytorch autograd version of calculating residual """
        u = self.net_u(x, y, t)
        
        u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
        
        u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
        u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]

       
        u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
        u_xxx = torch.autograd.grad(u_xx, x, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
        
        
        f_u = u_t + 3 * (u_x * u +  u * u_y) + u_xxx
        
        return f_u
    
    def loss_func(self):
        self.optimizer.zero_grad()

        u_pred = self.net_u(self.x_u, self.y_u, self.t_u)
        f_u_pred = self.net_f(self.x, self.y, self.t)

        loss_u = torch.mean((self.u - u_pred) ** 2)
        loss_f_u = torch.mean(f_u_pred ** 2)
        loss = loss_u + loss_f_u
        loss.backward()
        self.iter += 1
        self.loss_history.append(loss.item())
        if self.iter % 100 == 0:
            print('Iter %d, Loss_u: %.5e, Loss_f_u: %.5e'% (self.iter, loss_u.item(), loss_f_u.item()))

        return loss
    def loss_data(self):
        return self.loss_history
    
    def train(self, N_Adam_first):
        self.dnn.train()
        for i in range(N_Adam_first):
            self.optimizer = self.optimizer_Adam
            self.optimizer.step(self.loss_func)
        
        self.optimizer = self.optimizer
        self.optimizer.step(self.loss_func)
        
    def plot_loss(self):
        plt.semilogy(self.loss, label='loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()
        
    def predict(self, X):
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        y = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)
        t = torch.tensor(X[:, 2:3], requires_grad=True).float().to(device)

        self.dnn.eval()
        u = self.net_u(x, y, t)
        f_u = self.net_f(x, y, t)
        u = u.detach().cpu().numpy()
        f_u = f_u.detach().cpu().numpy()
        return u, f_u

## Exact solution

In [10]:
def exact_solution_u(x, y, z, t):
    alpha, beta = -1.2, 1
    lambda1, lambda2, lambda3, lambda4 = 1, 1, 1, 1
    k1, b1, k4, b4 = 1, 1, 1, 1
    xi = y + alpha * torch.tanh(y)
    theta = z + beta * torch.tanh(z)
    F2 = 1 / torch.cosh(xi) ** 2
    F3 = 1 / torch.cosh(theta) ** 2
    uu = lambda1 * (k1 * x + b1) + lambda2 * F2 + lambda3 * F3 + lambda4 * (k4 * t + b4)
    return uu

## Configurations

In [13]:
noise = 0.0
N_u = 100
N_f = 20000
layers = [4, 100, 100, 1]
# discrete scheme
Nx = 20
Ny = 20
Nz = 20
Nt = 20
x = np.linspace(-1, 1, Nx)
y = np.linspace(-1, 1, Ny)
z = np.linspace(-1, 1, Nz)
t = np.linspace(0, 1, Nt)

In [ ]:
X, Y, Z = np.meshgrid(x, y, z)
XYZ_ = np.hstack((X.flatten()[:,None],Y.flatten()[:,None], Y.flatten()[:,None]))
X, Y, Z = np.meshgrid(x, y, t)
XY_T = np.hstack((X.flatten()[:,None],Y.flatten()[:,None], Y.flatten()[:,None]))
X, Y, Z = np.meshgrid(x, z, t)
XZ_T = np.hstack((X.flatten()[:,None],Y.flatten()[:,None], Y.flatten()[:,None]))
X, Y, Z = np.meshgrid(y, z, t)
YZ_T = np.hstack((X.flatten()[:,None],Y.flatten()[:,None], Y.flatten()[:,None]))

uu1 = np.zeros((Nx * Ny * Nz, 1))

uu2 = np.zeros((Nx * Ny * Nt, 1))
uu3 = np.zeros((Nx * Ny * Nt, 1))

uu4 = np.zeros((Nx * Nz * Nt, 1))
uu5 = np.zeros((Nx * Nz * Nt, 1))

uu6 = np.zeros((Ny * Nz * Nt, 1))
uu7 = np.zeros((Ny * Nz * Nt, 1))


for i in range(Nx * Ny * Nz):
    uu1[i] = exact_solution_u(XYZ_[i,0], XYZ_[i,1], XYZ_[i,2], t[0]) # u(x,y,z,0)


for i in range(Nx * Ny * Nt):
    uu2[i] = exact_solution_u(XY_T[i,0], y[0], XY_T[i,1], XY_T[i,2]) # u(x,0,z,t)
    uu3[i] = exact_solution_u(XY_T[i,0], y[-1], XY_T[i,1], XY_T[i,2]) # u(x,1,z,t)
    
for i in range(Nx * Nz * Nt):
    uu4[i] = exact_solution_u(XZ_T[i,0], XZ_T[i,1], z[0], XZ_T[i,2]) # u(x,y,0,t)
    uu5[i] = exact_solution_u(XZ_T[i,0], XZ_T[i,1], z[-1], XZ_T[i,2]) # u(x,y,1,t)

for i in range(Ny * Nz * Nt):
    uu6[i] = exact_solution_u(x[0], YZ_T[i,0], YZ_T[i,1], YZ_T[i,2]) # u(0,y,z,t)
    uu7[i] = exact_solution_u(x[-1], YZ_T[i,0], YZ_T[i,1], YZ_T[i,2]) # u(1,y,z,t)

In [24]:
X, Y, Z, T = np.meshgrid(y, z, x, t)
X_star = np.hstack((X.flatten()[:,None], Y.flatten()[:,None], Z.flatten()[:,None],T.flatten()[:,None]))
u_star = np.zeros((X_star.shape[0], 1))
for i in range(X_star.shape[0]):
    u_star[i] = exact_solution_u(X_star[i, 0], X_star[i, 1], X_star[i, 2], X_star[i, 3])
    
# Doman bounds
lb = X_star.min(0)
ub = X_star.max(0)

In [ ]:
TTT = np.hstack((XYZ_, t[0] * np.ones((Nx * Ny * Nz,1))))
index = X_T.shape[1] // 2
YYY1 = np.insert(X_T, index, y[0], axis=1)
YYY2 = np.insert(X_T, index, y[-1],axis=1)
XXX1 = np.hstack((x[0] *np.ones((Nt * Ny,1)), _YT))
XXX2 = np.hstack((x[-1]*np.ones((Nt * Ny,1)), _YT))
X_u_train = np.vstack([TTT, YYY1, YYY2, XXX1, XXX2])


X_train = lb + (ub - lb) * lhs(3, N_f)
X_train = np.vstack((X_train, X_u_train))
u_train = np.vstack([uu1, uu2, uu3, uu4, uu5])
np.random.seed(seed)
idx = np.random.choice(X_u_train.shape[0], N_u, replace=False) # shape(100, )
X_u_train = X_u_train[idx,:]
u_train = u_train[idx,:]

## Training

In [ ]:
%%time
model = PhysicsInformedNN(X_u_train, u_train, X_train, layers, lb, ub)
model.train(0)

In [ ]:
u_pred = model.predict(X_star)
error_u = np.linalg.norm(u_star - u_pred, 2)/np.linalg.norm(u_star, 2)
print('Error u: %e' % (error_u))
U_pred = u_pred.reshape(Nx, Ny, Nt)
Exact_u = u_star.reshape(Nx, Ny, Nt)
Error_u = np.abs(Exact_u - U_pred)